In [1]:
import pandas as pd
import numpy as np
import shapely
import folium
from folium.plugins import HeatMap



In [2]:
country_data = pd.read_csv('./country_area/API_AG.LND.TOTL.K2_DS2_en_csv_v2_1036.csv')
city_data = pd.read_csv('./cities/worldcities.csv')

In [3]:
# For each country, pick the latest value of the area
country_data = country_data.dropna(subset=['2022'])
country_data = country_data[['Country Name', '2022']]

# Drop countries that don't appear in the city data
country_data = country_data[country_data['Country Name'].isin(city_data['country'])]
country_data.rename(columns={'Country Name': 'country', '2022': 'area'}, inplace=True)

city_data.dropna(subset=['population'], inplace=True)

In [4]:
def polygon_random_points (poly, num_points):
    min_x, min_y, max_x, max_y = poly.bounds
    points = []
    while len(points) < num_points:
        random_point = shapely.Point([np.random.uniform(min_x, max_x), np.random.uniform(min_y, max_y)])
        if (random_point.within(poly)):
            points.append(random_point)
    return points

In [14]:
def generate_country_data():
    # We first sample a country with a probability proportional to its surface area compared to the Earth’s total surface area.
    country = np.random.choice(country_data['country'], p=country_data['area']/country_data['area'].sum())
    # Randomly sample a city from the selected country
    picked_city_data = city_data[city_data['country'] == country]
    if picked_city_data.empty:
        return generate_country_data()
    # Randomly sample a city based on the population
    city_name = np.random.choice(picked_city_data['city'], p=picked_city_data['population']/picked_city_data['population'].sum())
    city = city_data[city_data['city'] == city_name].iloc[0]

    city_point = shapely.geometry.Point(city['lng'], city['lat'])
    city_buffer = city_point.buffer(0.25)

    return (country, city, polygon_random_points(city_buffer, 100))

In [15]:
m = folium.Map(zoom_start=2.2)

for i in range(100):
    country, city, points = generate_country_data()
    # folium.Marker([city['lat'], city['lng']], popup=city['city']).add_to(m)
    # for p in points:
    #     folium.Marker([p.y, p.x], icon=folium.Icon(color='green')).add_to(m)
    HeatMap([[p.y, p.x] for p in points], radius=20, blur=30).add_to(m)

m

In [8]:
import io
from PIL import Image

img_data = m._to_png(5)
img = Image.open(io.BytesIO(img_data))
img.save('image.png')